In [ ]:
# ============================================================
# SCRIPT COMPLETO (REFEITO): XGBoost por cidade + Tuning por cidade
# SEM SMOTE | COM TRATAMENTO DE COLUNAS CATEGÓRICAS (ONE-HOT)
#
# OBJETIVO:
# - Treinar 1 modelo XGBoost para cada cidade (coluna "localidade")
# - Respeitar o tempo de cada cidade (split temporal por cidade)
# - Fazer tuning de hiperparâmetros por cidade (Random Search)
# - Fazer validação temporal dentro do treino (TimeSeriesSplit)
# - Ajustar automaticamente desbalanceamento com scale_pos_weight
# - Escolher threshold no treino (opcional)
# - Avaliar no teste e salvar métricas
# - Exportar CSV final (1 linha por cidade) para usar no Power BI
#
# IMPORTANTE:
# - Este script evita o erro "dtype object" no XGBoost transformando
#   colunas texto/categóricas em one-hot (pd.get_dummies).
# - Também alinha colunas entre treino e teste, para não faltar dummy.
# ============================================================


# ----------------------------
# 1) IMPORTS (bibliotecas)
# ----------------------------

# Pandas: leitura e manipulação de tabelas
import pandas as pd

# Numpy: operações numéricas
import numpy as np

# JSON: para salvar hiperparâmetros como texto no CSV final
import json

# ParameterSampler: sorteia combinações aleatórias de hiperparâmetros (Random Search)
# TimeSeriesSplit: validação cruzada respeitando ordem temporal (sem embaralhar)
from sklearn.model_selection import ParameterSampler, TimeSeriesSplit

# Métricas: para avaliar desempenho de classificação
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score
)

# Modelo: XGBoost Classifier
from xgboost import XGBClassifier


# ============================================================
# 2) FUNÇÃO: limpar e preparar dataset (converte data e ordena)
# ============================================================

def preparar_dataset(df, col_data="data", col_cidade="localidade"):
    """
    Esta função:
    1) Faz uma cópia do DataFrame para não alterar o original.
    2) Converte a coluna de data para datetime.
    3) Remove linhas onde a data ficou inválida (NaT).
    4) Ordena por cidade e data para manter consistência temporal.
    5) Retorna o DataFrame pronto.
    """

    # Cria uma cópia do DataFrame original
    df_local = df.copy()

    # Converte a coluna de data para datetime; erros viram NaT
    df_local[col_data] = pd.to_datetime(df_local[col_data], errors="coerce")

    # Remove linhas que não têm data válida
    df_local = df_local.dropna(subset=[col_data])

    # Ordena por cidade e data para garantir ordem temporal
    df_local = df_local.sort_values([col_cidade, col_data]).reset_index(drop=True)

    # Retorna DataFrame pronto
    return df_local


# ============================================================
# 3) FUNÇÃO: gerar X e y de uma cidade + one-hot encoding
# ============================================================

def montar_X_y_com_onehot(df_cidade, feature_cols, col_target):
    """
    Esta função:
    1) Separa X (features) e y (target).
    2) Identifica colunas object (texto).
    3) Converte essas colunas para one-hot via get_dummies.
    4) Retorna X (já numérico) e y (int).
    """

    # X recebe apenas as colunas de features
    X = df_cidade[feature_cols].copy()

    # y recebe a coluna target e garante tipo int (0/1)
    y = df_cidade[col_target].astype(int).copy()

    # Identifica colunas de texto (dtype object)
    cols_obj = X.select_dtypes(include=["object"]).columns.tolist()

    # Se existirem colunas texto, faz one-hot nelas
    # dummy_na=True cria uma categoria para valores nulos (NaN)
    if len(cols_obj) > 0:
        X = pd.get_dummies(
            X,
            columns=cols_obj,
            dummy_na=True,
            drop_first=False
        )

    # Retorna X e y
    return X, y


# ============================================================
# 4) FUNÇÃO: split temporal (treino/teste) por cidade
# ============================================================

def split_temporal(X, y, proporcao_treino=0.8):
    """
    Esta função:
    1) Calcula um índice de corte baseado na proporção.
    2) Separa treino e teste respeitando a ordem temporal.
    3) Retorna X_train, y_train, X_test, y_test.
    """

    # Total de linhas
    n_total = len(X)

    # Índice de corte (por exemplo 80% do total)
    idx_corte = int(np.floor(proporcao_treino * n_total))

    # Treino recebe as primeiras linhas
    X_train = X.iloc[:idx_corte].reset_index(drop=True)
    y_train = y.iloc[:idx_corte].reset_index(drop=True)

    # Teste recebe as últimas linhas
    X_test = X.iloc[idx_corte:].reset_index(drop=True)
    y_test = y.iloc[idx_corte:].reset_index(drop=True)

    # Retorna conjuntos
    return X_train, y_train, X_test, y_test


# ============================================================
# 5) FUNÇÃO: alinhar colunas entre treino e teste (pós one-hot)
# ============================================================

def alinhar_colunas_treino_teste(X_train, X_test):
    """
    Esta função garante que:
    - X_train e X_test tenham exatamente as mesmas colunas.
    - Se uma coluna existir em um e não no outro, ela é criada com valor 0.
    """

    # align(..., join="left") garante que X_test tenha pelo menos as colunas do treino
    # fill_value=0 preenche colunas ausentes com zeros
    X_train_al, X_test_al = X_train.align(X_test, join="left", axis=1, fill_value=0)

    # Retorna alinhados
    return X_train_al, X_test_al


# ============================================================
# 6) FUNÇÃO: calcular scale_pos_weight (desbalanceamento)
# ============================================================

def calcular_scale_pos_weight(y):
    """
    scale_pos_weight = (n_negativos / n_positivos)
    Isso ajuda o XGBoost a não ignorar a classe minoritária.
    """

    # Conta negativos e positivos
    neg = int((y == 0).sum())
    pos = int((y == 1).sum())

    # Evita divisão por zero
    if pos == 0:
        return 1.0

    # Retorna a razão
    return float(neg / pos)


# ============================================================
# 7) FUNÇÃO: calcular métricas no teste
# ============================================================

def calcular_metricas_classificacao(y_true, y_proba, threshold=0.5):
    """
    Recebe:
    - y_true: labels reais (0/1)
    - y_proba: probabilidades previstas para classe 1
    - threshold: corte para virar classe

    Retorna:
    - métricas (precision, recall, f1, roc_auc, pr_auc)
    - TP/TN/FP/FN
    """

    # Converte probabilidade em classe com o threshold
    y_pred = (y_proba >= threshold).astype(int)

    # Matriz de confusão: TN, FP, FN, TP
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # Métricas baseadas em classe
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # Métricas baseadas em probabilidade (precisa ter as duas classes)
    if len(np.unique(y_true)) == 2:
        roc_auc = roc_auc_score(y_true, y_proba)
        pr_auc = average_precision_score(y_true, y_proba)
    else:
        roc_auc = np.nan
        pr_auc = np.nan

    # Retorna dicionário de métricas
    return {
        "threshold": float(threshold),
        "tp": int(tp),
        "tn": int(tn),
        "fp": int(fp),
        "fn": int(fn),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "roc_auc": float(roc_auc) if roc_auc == roc_auc else np.nan,
        "pr_auc": float(pr_auc) if pr_auc == pr_auc else np.nan
    }


# ============================================================
# 8) FUNÇÃO: escolher threshold no treino (opcional)
# ============================================================

def escolher_threshold_no_treino(y_true, y_proba, metrica="f1"):
    """
    Esta função tenta vários thresholds e escolhe o melhor baseado em:
    - f1 (padrão), ou
    - recall, ou
    - precision

    Ela testa thresholds de 0.10 até 0.90, passo 0.05
    """

    # Cria lista de thresholds
    thresholds = np.round(np.arange(0.10, 0.91, 0.05), 2)

    # Define melhor threshold e melhor valor
    melhor_thr = 0.5
    melhor_val = -np.inf

    # Testa cada threshold
    for thr in thresholds:
        # Calcula métricas com esse threshold
        m = calcular_metricas_classificacao(y_true, y_proba, threshold=thr)

        # Pega a métrica escolhida
        val = m.get(metrica, np.nan)

        # Atualiza se for melhor
        if val == val and val > melhor_val:
            melhor_val = val
            melhor_thr = thr

    # Retorna melhor threshold encontrado
    return float(melhor_thr)


# ============================================================
# 9) FUNÇÃO: avaliar hiperparâmetros com validação temporal
# ============================================================

def avaliar_params_com_validacao_temporal(X_train, y_train, params, n_splits=4, scoring="pr_auc", random_state=42):
    """
    Para um conjunto de hiperparâmetros (params), esta função:
    1) Divide X_train/y_train em folds temporais com TimeSeriesSplit.
    2) Treina em cada fold de treino e avalia no fold de validação.
    3) Calcula a métrica escolhida (scoring) e guarda.
    4) Retorna a média dos scores.
    """

    # Cria objeto de validação temporal
    tss = TimeSeriesSplit(n_splits=n_splits)

    # Lista de scores de cada fold
    scores = []

    # Para cada divisão temporal
    for idx_tr, idx_val in tss.split(X_train):
        # Separa treino do fold
        X_tr = X_train.iloc[idx_tr]
        y_tr = y_train.iloc[idx_tr]

        # Separa validação do fold
        X_val = X_train.iloc[idx_val]
        y_val = y_train.iloc[idx_val]

        # Se treino ou validação tiver apenas 1 classe, pule o fold
        if len(np.unique(y_tr)) < 2 or len(np.unique(y_val)) < 2:
            continue

        # Calcula scale_pos_weight no fold de treino
        spw = calcular_scale_pos_weight(y_tr)

        # Cria modelo com params do trial
        model = XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=random_state,
            n_jobs=-1,
            scale_pos_weight=spw,
            **params
        )

        # Treina no fold de treino
        model.fit(X_tr, y_tr)

        # Prediz probabilidade no fold de validação
        y_proba_val = model.predict_proba(X_val)[:, 1]

        # Calcula score conforme métrica escolhida
        if scoring == "pr_auc":
            score = average_precision_score(y_val, y_proba_val)
        elif scoring == "roc_auc":
            score = roc_auc_score(y_val, y_proba_val)
        else:
            score = average_precision_score(y_val, y_proba_val)

        # Guarda score desse fold
        scores.append(score)

    # Se não houve fold válido, retorne -inf (inválido)
    if len(scores) == 0:
        return -np.inf

    # Retorna média dos scores
    return float(np.mean(scores))


# ============================================================
# 10) FUNÇÃO PRINCIPAL: treinar por cidade com tuning
# ============================================================

def treinar_xgb_por_cidade_com_tuning(
    df,
    col_cidade="localidade",
    col_data="data",
    col_target="chove_amanha_vtr",
    proporcao_treino=0.8,
    min_linhas_cidade=500,
    n_iter=50,
    n_splits_tss=4,
    scoring_valid="pr_auc",
    usar_threshold_otimo=True,
    metrica_threshold="f1",
    random_state=42
):
    """
    Função principal que:
    - prepara dataset (data datetime + ordenação)
    - itera por cidade
    - monta X e y com one-hot
    - split temporal
    - tuning com random search
    - treina modelo final com melhores params
    - escolhe threshold no treino (opcional)
    - avalia no teste
    - salva 1 linha de métricas por cidade
    """

    # Prepara dataset (data datetime e ordenação)
    df_local = preparar_dataset(df, col_data=col_data, col_cidade=col_cidade)

    # Define colunas que não são features
    colunas_nao_features = {col_cidade, col_data, col_target}

    # Define lista de features como todas as colunas exceto as não-features
    feature_cols = [c for c in df_local.columns if c not in colunas_nao_features]

    # Define espaço de hiperparâmetros (realista, sem explodir)
    param_distributions = {
        "n_estimators": [200, 400, 600, 800],
        "learning_rate": [0.01, 0.03, 0.05, 0.1],
        "max_depth": [2, 3, 4, 5, 6],
        "subsample": [0.7, 0.8, 0.9, 1.0],
        "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
        "min_child_weight": [1, 3, 5, 7, 10],
        "reg_lambda": [1.0, 2.0, 5.0, 10.0],
        "gamma": [0, 0.5, 1.0, 2.0]
    }

    # Gera combinações aleatórias de hiperparâmetros
    sampler = ParameterSampler(
        param_distributions=param_distributions,
        n_iter=n_iter,
        random_state=random_state
    )

    # Lista para guardar resultados finais
    resultados = []

    # Lista de cidades únicas
    cidades = df_local[col_cidade].dropna().unique()

    # Itera por cidade
    for cidade in cidades:
        # Filtra dados da cidade
        df_cid = df_local[df_local[col_cidade] == cidade].copy()

        # Conta total de linhas
        n_total = int(len(df_cid))

        # Se tiver poucas linhas, marca e continua
        if n_total < min_linhas_cidade:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": f"n_total < {min_linhas_cidade}",
                "n_total": n_total
            })
            continue

        # Ordena por data dentro da cidade
        df_cid = df_cid.sort_values(col_data).reset_index(drop=True)

        # Monta X e y já com one-hot
        X, y = montar_X_y_com_onehot(df_cid, feature_cols=feature_cols, col_target=col_target)

        # Se total tiver apenas uma classe, não dá para treinar
        if len(np.unique(y)) < 2:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "apenas_uma_classe_no_total",
                "n_total": n_total
            })
            continue

        # Split temporal
        X_train, y_train, X_test, y_test = split_temporal(X, y, proporcao_treino=proporcao_treino)

        # Alinha colunas de treino e teste (pós one-hot)
        X_train, X_test = alinhar_colunas_treino_teste(X_train, X_test)

        # Se treino ou teste tiver apenas uma classe, não dá para avaliar
        if len(np.unique(y_train)) < 2 or len(np.unique(y_test)) < 2:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "treino_ou_teste_sem_duas_classes",
                "n_total": n_total,
                "n_treino": int(len(y_train)),
                "n_teste": int(len(y_test))
            })
            continue

        # Variáveis para melhor resultado
        melhor_score = -np.inf
        melhor_params = None

        # Tuning: testa combinações de hiperparâmetros
        for trial_id, params in enumerate(sampler):
            score = avaliar_params_com_validacao_temporal(
                X_train=X_train,
                y_train=y_train,
                params=params,
                n_splits=n_splits_tss,
                scoring=scoring_valid,
                random_state=random_state
            )

            # Atualiza se for melhor
            if score > melhor_score:
                melhor_score = score
                melhor_params = params

        # Se não achou trial válido
        if melhor_params is None or melhor_score == -np.inf:
            resultados.append({
                "cidade": cidade,
                "cidade_insuficiente": True,
                "motivo": "nenhum_trial_valido",
                "n_total": n_total
            })
            continue

        # Calcula scale_pos_weight no treino inteiro
        spw_final = calcular_scale_pos_weight(y_train)

        # Cria modelo final com melhores parâmetros
        model_final = XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=random_state,
            n_jobs=-1,
            scale_pos_weight=spw_final,
            **melhor_params
        )

        # Treina modelo final
        model_final.fit(X_train, y_train)

        # Probabilidade no treino (para escolher threshold)
        y_proba_train = model_final.predict_proba(X_train)[:, 1]

        # Probabilidade no teste (para métricas finais)
        y_proba_test = model_final.predict_proba(X_test)[:, 1]

        # Escolhe threshold no treino (opcional)
        if usar_threshold_otimo:
            thr_escolhido = escolher_threshold_no_treino(
                y_true=y_train,
                y_proba=y_proba_train,
                metrica=metrica_threshold
            )
        else:
            thr_escolhido = 0.5

        # Calcula métricas no teste
        metricas_teste = calcular_metricas_classificacao(
            y_true=y_test,
            y_proba=y_proba_test,
            threshold=thr_escolhido
        )

        # Informações temporais: ano início e ano fim
        ano_inicio = int(df_cid[col_data].dt.year.min())
        ano_fim = int(df_cid[col_data].dt.year.max())

        # Prevalência de chuva no total e no teste
        preval_total = float(y.mean())
        preval_teste = float(y_test.mean())

        # Tamanho do teste
        n_teste = int(len(y_test))

        # Nível de confiabilidade baseado no tamanho do teste
        if n_teste < 200:
            confiabilidade = "baixa"
        elif n_teste < 800:
            confiabilidade = "media"
        else:
            confiabilidade = "alta"

        # Número final de features (após one-hot)
        n_features = int(X_train.shape[1])

        # Salva resultado final da cidade
        resultados.append({
            "cidade": cidade,
            "cidade_insuficiente": False,
            "motivo": "",
            "ano_inicio": ano_inicio,
            "ano_fim": ano_fim,
            "n_total": n_total,
            "n_treino": int(len(y_train)),
            "n_teste": n_teste,
            "n_features": n_features,
            "prevalencia_total": preval_total,
            "prevalencia_teste": preval_teste,
            "confiabilidade": confiabilidade,
            "scoring_valid": scoring_valid,
            "best_valid_score": float(melhor_score),
            "best_params_json": json.dumps(melhor_params, ensure_ascii=False),
            **metricas_teste
        })

    # Converte lista de resultados em DataFrame
    df_resultados = pd.DataFrame(resultados)

    # Retorna DataFrame final
    return df_resultados


# ============================================================
# 11) EXECUÇÃO (MAIN)
# ============================================================

if __name__ == "__main__":
    # --------------------------------------------------------
    # A) Defina o caminho do arquivo Excel
    # --------------------------------------------------------
    # IMPORTANTE:
    # - Use r"" para evitar problemas com barras invertidas no Windows.
    # - Corrija o caminho conforme seu PC.
    # --------------------------------------------------------

    caminho_excel = r"C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v9.xlsx"

    # --------------------------------------------------------
    # B) Leia o Excel para um DataFrame
    # --------------------------------------------------------

    df = pd.read_excel(caminho_excel)

    # --------------------------------------------------------
    # C) Rode o treinamento por cidade com tuning
    # --------------------------------------------------------

    df_metricas_cidades = treinar_xgb_por_cidade_com_tuning(
        df=df,
        col_cidade="localidade",
        col_data="data",
        col_target="chove_amanha_vtr",
        proporcao_treino=0.8,
        min_linhas_cidade=500,
        n_iter=50,               # Aumente para 100+ se quiser mais busca (demora mais)
        n_splits_tss=4,
        scoring_valid="pr_auc",  # bom para desbalanceamento
        usar_threshold_otimo=True,
        metrica_threshold="f1",
        random_state=42
    )

    # --------------------------------------------------------
    # D) Salve o resultado final em CSV (para Power BI)
    # --------------------------------------------------------

    #saida_csv = r"C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\metricas_por_cidade_xgb_v1.csv"
    #df_metricas_cidades.to_csv(saida_csv, index=False)

    # --------------------------------------------------------
    # E) Mostre as primeiras linhas no console
    # --------------------------------------------------------

    print(df_metricas_cidades)


OSError: Cannot save file into a non-existent directory: 'C:\Users\JacyzinGuilherme(Bip)\mentoria-bip\dados_editados'